<a href="https://colab.research.google.com/github/lcerda7/HerramientasComputacionales/blob/main/Clases/11_7_22_Python%2C_weather%2C_crime%2C_etc_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/ekapope/WorldWeatherOnline

API key generado haciéndose una cuenta en https://www.worldweatheronline.com/ - free trial por 30 días con 500 requests por día!

Si les salta error es porque ya se gastó por hoy! Hacerse una cuenta para usarlo (uno por pareja - lo van a necesitar para la tarea).


In [ ]:
# From datetime module import datetime class
from datetime import datetime

In [ ]:
# We then used now() method to create a datetime object containing the current local date and time.
start_time = datetime.now()
print(start_time)

In [ ]:
import pandas as pd
start_date = '11-DEC-2018'
end_date = '31-DEC-2019'    
    
# create list of first day of month for range between start and end dates non-inclusive (open)
# MS= month start frequency
list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')

# .to_datetime converts argument to datetime
list_mon_begin1 = [pd.Series(pd.to_datetime(start_date))]
list_mon_begin2 = [pd.Series(list_mon_begin)]

print(list_mon_begin)

In [ ]:
print(list_mon_begin1)
print(list_mon_begin2)

In [ ]:
# convert to Series and add start_date at beginning
list_mon_begin = pd.concat([pd.Series(pd.to_datetime(start_date)), pd.Series(list_mon_begin)], ignore_index=True)
print(list_mon_begin)

In [ ]:
# create list of month end dates for range between start and end dates non-inclusive (open)
# month end frequency
list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')
# convert to Series and add end_date at end
list_mon_end = pd.concat([pd.Series(list_mon_end), pd.Series(pd.to_datetime(end_date))], ignore_index=True)

print(list_mon_end)

In [ ]:
# count number of months to be retrieved
total_months = len(list_mon_begin)
print(total_months)

In [ ]:
# initialize df_hist to store return data
df_hist = pd.DataFrame()

In [ ]:
frequency=24
api_key = '535a0388efce401796922753220807'
location_list = ['New+York']
response_cache_path=None
location='New+York'

In [ ]:
import urllib
import json
import os

JSON is a syntax for storing and exchanging data.
JSON is text, written with JavaScript object notation.

If you have a JSON string, you can parse it by using the `json.loads()` method. **The result will be a Python dictionary.**

If you have a Python object, you can convert it into a JSON string by using the `json.dumps()` method.

In [ ]:
import json
# some JSON:
x =  '{ "name":"John", "age":30, "city":"New York"}'
# parse x:
y = json.loads(x)
# the result is a Python dictionary:
print(y["age"])

In [ ]:
print(y)

In [ ]:
import json
x = {
  "name": "John",
  "age": 30,
  "married": True,
  "divorced": False,
  "children": ("Ann","Billy"),
  "pets": None,
  "cars": [
    {"model": "BMW 230", "mpg": 27.5},
    {"model": "Ford Edge", "mpg": 24.1}
  ]
}
# convert into JSON:
y = json.dumps(x)
# the result is a JSON string:
print(y)

`urllib` is a package that collects several modules for working with URLs ([Help](https://docs.python.org/3/library/urllib.html)):
* `urllib.request` for opening and reading URLs
* `urllib.error` containing the exceptions raised by urllib.request
* `urllib.parse` for parsing URLs
* `urllib.robotparser` for parsing robots.txt files

In [ ]:
import urllib.request
with urllib.request.urlopen('http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=535a0388efce401796922753220807&q=New+York&format=json&date=2018-12-11&enddate=2018-12-12&tp=24') as response:
   html = response.read()

print(html)

OS module in Python provides functions for interacting with the operating system. 

`os.path.exists()` method in Python is used to check whether the specified path exists or not. This method can be also used to check whether the given path refers to an open file descriptor or not.

Return Type: This method returns a Boolean value of class bool. This method returns True if path exists otherwise returns False.

# Seguimos...

In [ ]:
for m in range(total_months):
        start_d = str(list_mon_begin[m])[:10]
        end_d = str(list_mon_end[m])[:10]
        print(f'Esto es m {m}')
        print(f'Esto es start_d {start_d}')
        print(f'Esto es end_d {end_d}')


In [ ]:
    for m in range(total_months):
        start_d = str(list_mon_begin[m])[:10]
        end_d = str(list_mon_end[m])[:10]
        file_path = f'{response_cache_path}/{location}_{start_d}_{end_d}'
        if response_cache_path and os.path.exists(file_path):
            print('Reading cached data for ' + location + ': from ' + start_d + ' to ' + end_d)
            with open(f'{response_cache_path}/{location}_{start_d}_{end_d}', 'r') as f:
                json_data = json.load(f)
        else:
            print('Currently retrieving data for ' + location + ': from ' + start_d + ' to ' + end_d)
            url_page = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=' + api_key + '&q=' + location + '&format=json&date=' + start_d + '&enddate=' + end_d + '&tp=' + str(
                frequency)
            json_page = urllib.request.urlopen(url_page, timeout=10)
            json_data = json.loads(json_page.read().decode())
            

In [ ]:
print(json_data)

In [ ]:
        if response_cache_path:
            with open(f'{response_cache_path}/{location}_{start_d}_{end_d}', 'w') as f:
                json.dump(json_data, f)
        data = json_data['data']['weather']
        print(data)

In [ ]:
num_days = len(data)
# initialize df_month to store return data
df_month = pd.DataFrame()
for i in range(num_days):
    # extract this day
    d = data[i]
    # astronomy data is the same for the whole day
    astr_df = pd.DataFrame(d['astronomy'])
    # hourly data; temperature for each hour of the day
    hourly_df = pd.DataFrame(d['hourly'])
    # this wanted_key will be duplicated and use 'ffill' to fill up the NAs
    wanted_keys = ['date', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex']  # The keys you want
    subset_d = dict((k, d[k]) for k in wanted_keys if k in d)
    this_df = pd.DataFrame(subset_d, index=[0])
    df = pd.concat([this_df.reset_index(drop=True), astr_df], axis=1)
    # concat selected astonomy columns with hourly data
    df = pd.concat([df, hourly_df], axis=1)
    df = df.fillna(method='ffill')
    # make date_time columm to proper format
    # fill leading zero for hours to 4 digits (0000-2400 hr)
    df['time'] = df['time'].apply(lambda x: x.zfill(4))
    # keep only first 2 digit (00-24 hr) 
    df['time'] = df['time'].str[:2]
    # convert to pandas datetime
    df['date_time'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    # keep only interested columns
    col_to_keep = ['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex',
                   'moon_illumination', 'moonrise', 'moonset', 'sunrise', 'sunset',
                   'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph',
                   'cloudcover', 'humidity', 'precipMM', 'pressure', 'tempC', 'visibility',
                   'winddirDegree', 'windspeedKmph']
    df = df[col_to_keep]
    df = df.loc[:,~df.columns.duplicated()]
    df_month = pd.concat([df_month, df])

In [ ]:
print(df_month)

In [ ]:
# call function to extract json object
df_this_month = df_month
df_this_month['location'] = location
df_hist = pd.concat([df_hist, df_this_month])

print(df_hist)